In [ ]:
import pandas as pd
import numpy as np
import glob
import re
%matplotlib inline
import bs4
from bs4 import BeautifulSoup
import csv

In [ ]:
# Open data and convert it to pandas
data=[]
with open('../data/class-data-with-all-tests-and-concept-check-NO_IDENTIFIERS.csv', mode='r') as file:
    csv_reader = csv.reader(file)
    data = list(csv_reader)
    
headers = data[0]
rest = data[1:]

everything = pd.DataFrame(rest, columns=headers)

In [ ]:
def as_int(dat):
    return dat.apply(lambda x: int(x) if x else 0)
def score1a(data):
    return (data['1a'] == 'c').astype(int)
def score1b(data):
    a,b,c,d,e,f = as_int(data['1b.a']),as_int(data['1b.b']),as_int(data['1b.c']),as_int(data['1b.d']),as_int(data['1b.e']),as_int(data['1b.f'])
    return (a+(1-b)+c+(1-d)+e+(1-f))/6
def score2a(data):
    return (data['2a'] == 'a').astype(int)
def score2b(data):
    return (data['2b'] == '0').astype(int)
def score2c(data):
    a,b,c,d,e = map(as_int, (data['2c.a'],data['2c.b'],data['2c.c'],data['2c.d'],data['2c.e']))
    return ((1-a)+b+(1-c)+(1-d)+(1-e))/5
def score3(data):
    a,b,c,d,e,f = map(as_int, (data['3.a'],data['3.b'],data['3.c'],data['3.d'],data['3.e'],data['3.f']))
    return ((1-a)+b+c+d+(1-e)+f)/6
def score4a(data):
    return (data['4a'] == 'd').astype(int)
def score5(data):
    a,b,c,d,e = map(as_int, (data['5.a'],data['5.b'],data['5.c'],data['5.d'],data['5.e']))
    return (a+(1-b)+(1-c)+d+(1-e))/5
def score6a(data):
    return (data['6a'] == '1').astype(int)
def score6b(data):
    return (data['6b'] == 'c').astype(int)
def score7(data):
    return (data['7'] == 'd').astype(int)
def score8(data):
    a,b,c,d,e,f = map(as_int, (data['8.a'],data['8.b'],data['8.c'],data['8.d'],data['8.e'],data['8.f']))
    return ((1-a)+(1-b)+(1-c)+(1-d)+e+(1-f))/6
def score9a(data):
    a,b,c,d,e,f,g,h,i = map(as_int, (data['9a.a'],data['9a.b'],data['9a.c'],data['9a.d'],data['9a.e'],data['9a.f'],data['9a.g'],data['9a.h'],data['9a.i']))
    return (a+(1-b)+c+d+(1-e)+f+(1-g)+(1-h)+(1-i))/9
def score9b(data):
    a,b,c,d,e,f = map(as_int, (data['9b.a'],data['9b.b'],data['9b.c'],data['9b.d'],data['9b.e'],data['9b.f']))
    return ((1-a)+(1-b)+(1-c)+(1-d)+e+(1-f))/6
def score10(data):
    a,b,c,d,e = map(as_int, (data['10.a'],data['10.b'],data['10.c'],data['10.d'],data['10.e']))
    return ((1-a)+(1-b)+c+d+e)/5

In [ ]:
assessment = pd.DataFrame({"1a":score1a(everything),"1b":score1b(everything),"2a":score2a(everything),
                           "2b":score2b(everything),"2c":score2c(everything),"3":score3(everything),
                           "4a":score4a(everything),"5":score5(everything),"6a":score6a(everything),
                           "6b":score6b(everything),"7":score7(everything),"8":score8(everything),
                          "9a":score9a(everything),"9b":score9b(everything),"10":score10(everything)})

assessment.describe().loc[['mean','std']].round(2)

# Keras Computations

Begin by formatting our data before feeding it into a densely connected 3 layer network. 

In [ ]:
train_size = 264

# Convert dataframe back to numpy array
assessement_np = assessment.to_numpy()

# Separate training data from testing data and grab labels
train_data = assessement_np[:train_size]
train_labels = everything['Final.total'].to_numpy()[:train_size].astype(np.float)

test_data = train_data
test_labels = train_labels

#test_data = assessement_np[train_size:]
#test_labels = everything['Final.total'].to_numpy()[train_size:].astype(np.float)

In [ ]:
# Import Keras
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

# Build the model.
model = Sequential([
  Dense(1, activation='relu', input_shape=(15,))
])

# Compile the model.
model.compile(
  optimizer='adam',
  loss='mean_absolute_error',
  metrics=['mean_absolute_error'],
)

# Train the model.
model.fit(
  train_data,
  train_labels,
  epochs=1000,
  batch_size=32,
)

#Evaluate the model.
model.evaluate(
    test_data,
    test_labels
)

# Save the model to disk.
#model.save_weights('model.h5')

# Load the model from disk later using:
# model.load_weights('model.h5')

# Predict on the first 5 test images.
#predictions = model.predict(test_images[:5])

# Print our model's predictions.
#print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]

# Check our predictions against the ground truths.
#print(test_labels[:5]) # [7, 2, 1, 0, 4]

In [ ]:
# Check our predications against the true values
test_data = assessement_np[21:25]
model.predict(test_data)

In [ ]:
print(everything['Final.total'].to_numpy()[21:25].astype(np.float))

In [ ]:
# Grab individual weights of the network
model.layers[0].get_weights()[0]

To-Do 
1. Investigate and break the model in the ways we know it should
2. Implement new nets for the individual select all the apply questions
3. Check that midterm 1 and 2 are both roughly equivalent indicators of final exam scores